# ECE_7500
##Assignment 1
## Jaideep Siva Senthil



In [30]:
#importing packages
import pandas as pd
import regex as re
import string

# Dataset importing

In [64]:
#importing dataset

url = 'https://raw.githubusercontent.com/jaideep-siva/Assignment_1/main/spam_ham_dataset.csv'
df = pd.read_csv(url)


#Data analysis

In [65]:
# Dimensions of the data set
df.shape

(5171, 4)

In [66]:
# sample values from data set
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


> It appears that data frame consists of 4 columns with 2 label coluns, one text column and one column with count.

In [29]:
df['label'].value_counts()

ham     3672
spam    1499
Name: label, dtype: int64

> There appears to be a class imbalance with hame emails having twice as more data points as spam emails

In [32]:
#checking for nonexistent values
df.isna().sum()

Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64

#Data preprocessing

In [72]:
class data_preprocessing:

  def __init__(self,dataframe):
    self.data = df


  def lower_text(self,text):
    # Lower casing all text so the values are easily processed
    text = text.lower()
    return text

  def remove_subject_re(self,text):
    # Removing "Subject:" and  "Re:" from emails since they appear to have no inherent value and appear in all emails
    text = re.sub(r'^\s*subject:\s*(re\s*:)?\s*', '', text)
    return text

  def remove_numbers(self,text):
    # removing numbers, from looking at most of the emails, dates and random number values do not seem to attribute any value
    text = re.sub(r'[0-9]+', '', text)
    return text

  def remove_punctuations(self,text):
    # Removing punctuations since loads of random charecters can become tokens making the text corpus very massive for computation
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [76]:
# lowercasing all text in df
df['text'] = df['text'].apply(lambda x: data_preprocessing(df).lower_text(x))

df.head()


,Unnamed: 0,label,text,label_num
0,605,ham,enron methanol ; meter # : 988291\r\nthis is a...,0
1,2349,ham,"hpl nom for january 9 , 2001\r\n( see attached...",0
2,3624,ham,"neon retreat\r\nho ho ho , we ' re around to t...",0
3,4685,spam,"photoshop , windows , office . cheap . main tr...",1
4,2030,ham,indian springs\r\nthis deal is to book the tec...,0


In [77]:
# removing Subject: and re: from text
df['text'] = df['text'].apply(lambda x: data_preprocessing(df).remove_subject_re(x))

df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,enron methanol ; meter # : 988291\r\nthis is a...,0
1,2349,ham,"hpl nom for january 9 , 2001\r\n( see attached...",0
2,3624,ham,"neon retreat\r\nho ho ho , we ' re around to t...",0
3,4685,spam,"photoshop , windows , office . cheap . main tr...",1
4,2030,ham,indian springs\r\nthis deal is to book the tec...,0


In [78]:
df['text'] = df['text'].apply(lambda x: data_preprocessing(df).remove_numbers(x))
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,enron methanol ; meter # : \r\nthis is a follo...,0
1,2349,ham,"hpl nom for january , \r\n( see attached file...",0
2,3624,ham,"neon retreat\r\nho ho ho , we ' re around to t...",0
3,4685,spam,"photoshop , windows , office . cheap . main tr...",1
4,2030,ham,indian springs\r\nthis deal is to book the tec...,0


In [80]:
df['text'] = df['text'].apply(lambda x: data_preprocessing(df).remove_punctuations(x))
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,enron methanol meter \r\nthis is a follow u...,0
1,2349,ham,hpl nom for january \r\n see attached file ...,0
2,3624,ham,neon retreat\r\nho ho ho we re around to tha...,0
3,4685,spam,photoshop windows office cheap main trendi...,1
4,2030,ham,indian springs\r\nthis deal is to book the tec...,0


#Feature Extraction

## Class and functions for feature extraction

In [95]:
class feature_extraction:

  def __init__(self,df):

    self.data = df
    self.frequency ={}

  def frequency(self):


      for index, row in self.data.iterrows():
        words = row['text'].split()
        label = row['label']

        """
        For every word in words list
        if the word has a label of ham add it to the ham frequency dict
        else add it to the spam dict
       """

        for word in words:
          if (word, label) in self.frequency:
            self.frequency[(word, label)] += 1
          else:
            self.frequency[(word, label)] = 1

  def extract_features(self, text):
        words = text.split()
        bias = 1  # Bias term
        spam_frequency = 0
        ham_frequency = 0
        for word in words:
            spam_frequency += self.frequency.get((word, 'spam'), 0)
            ham_frequency += self.frequency.get((word, 'ham'), 0)
        return [bias, spam_frequency, ham_frequency]




## Getting features

In [97]:
freq = feature_extraction(df)
freq.frequency()
df['features'] = df['text'].apply(lambda x: freq.feature_extraction(x))
df.head()



TypeError: ignored